In [5]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [6]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
190,I almost made a fool of myself when I was goin...,negative
663,Moonwalker is probably not the film to watch i...,positive
44,I watched this last night for the first time i...,negative
386,"I haven't read a biography of Lincoln, so mayb...",negative
681,Definitely one of the most witty and twisted w...,positive


In [8]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13008\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [9]:
df = normalize_text(df)
df.head()

,review,sentiment
190,almost made fool going start review saying mov...,negative
663,moonwalker probably film watch michael jackson...,positive
44,watched last night first time year bad really ...,negative
386,read biography lincoln maybe accurate portraya...,negative
681,definitely one witty twisted who dunnit ever s...,positive


In [10]:
df['sentiment'].value_counts()

sentiment
negative    257
positive    243
Name: count, dtype: int64

In [11]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [12]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
190,almost made fool going start review saying mov...,0
663,moonwalker probably film watch michael jackson...,1
44,watched last night first time year bad really ...,0
386,read biography lincoln maybe accurate portraya...,0
681,definitely one witty twisted who dunnit ever s...,1


In [13]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
import mlflow
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/madhusudhanreddy8074/capstone-project1.mlflow')
dagshub.init(repo_owner='madhusudhanreddy8074', repo_name='Capstone-Project1', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "madhusudhanreddy8074/Capstone-Project1"

Repository madhusudhanreddy8074/Capstone-Project1 initialized!

<Experiment: artifact_location='mlflow-artifacts:/15a58835f9a64bdb839030c068cfe6c9', creation_time=1766028118089, experiment_id='0', last_update_time=1766028118089, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [19]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-12-18 09:02:34,988 - INFO - Starting MLflow run...
2025-12-18 09:02:37,727 - INFO - Logging preprocessing parameters...
2025-12-18 09:02:39,090 - INFO - Initializing Logistic Regression model...
2025-12-18 09:02:39,099 - INFO - Fitting the model...
2025-12-18 09:02:39,316 - INFO - Model training complete.
2025-12-18 09:02:39,323 - INFO - Logging model parameters...
2025-12-18 09:02:39,764 - INFO - Making predictions...
2025-12-18 09:02:39,768 - INFO - Calculating evaluation metrics...
2025-12-18 09:02:39,881 - INFO - Logging evaluation metrics...
2025-12-18 09:02:41,671 - INFO - Saving and logging the model...
2025/12/18 09:02:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-18 09:03:27,708 - INFO - Model training and logging completed in 49.98 seconds.
2025-12-18 09:03:27,714 - INFO - Accuracy: 0.72
2025-12-18 09:03:27,717 - INFO - Precision: 0.6811594202898551
2025-12-18 09:03:27,719 - INFO - Recall: 0.7833333333333333
2025-12-18 

🏃 View run mysterious-goat-13 at: https://dagshub.com/madhusudhanreddy8074/capstone-project1.mlflow/#/experiments/0/runs/4ea8512a210645e88de10744ed91cd5e
🧪 View experiment at: https://dagshub.com/madhusudhanreddy8074/capstone-project1.mlflow/#/experiments/0
